# Optimisation



In [1]:
%reset -f

In [3]:
import jax
from jax import vmap,grad
import jax.numpy as jnp
import os
import matplotlib.pyplot as plt

## Fonctions exemples

### Ploter

In [70]:
def plot_function(ax,fn,x_range=[-2,2],y_range=[-2,2]):

    x=jnp.linspace(x_range[0],x_range[1],100)
    y=jnp.linspace(y_range[0],y_range[1],100)

    X,Y=jnp.meshgrid(x,y,indexing="ij")
    XY_flat=jnp.stack([X.flatten(),Y.flatten()],axis=1)


    Z=vmap(fn)(XY_flat).reshape(X.shape)

    ax.pcolormesh(X,Y,Z,cmap="jet",shading="gouraud")

### Fonction oscillante

In [7]:
def oscillating(xy):
    x0=xy[0]
    x1=xy[1]
    return jnp.sin(3*x0)*jnp.cos(3*x1)+(x0+x1)/4

fig,ax=plt.subplots()
plot_function(ax,oscillating)

### Des bols bizarres

In [72]:
def elongated_bowl(xy):
    x0=xy[0]
    x1=xy[1]
    return 50*x0**2+2*x1**2

In [73]:
def leaking_bowl(xy):
    x0=xy[0]
    x1=xy[1]
    return 50*x0**2+2*x1**3

In [74]:
fig,(ax0,ax1)=plt.subplots(1,2)
plot_function(ax0,elongated_bowl)
plot_function(ax1,leaking_bowl)

### SGD

In [75]:
import optax

In [76]:
def apply_optimizer(fn,inp0,opt,nb):

    opt_state=opt.init(inp0)

    inps=[inp0]
    inp=inp0
    for _ in range(nb):
        grad=jax.grad(fn)(inp)
        update,opt_state=opt.update(grad,opt_state,inp)
        inp=inp+update

        inps.append(inp)

    return jnp.stack(inps)

In [77]:
lr=0.1
fn=oscillating
opt=optax.sgd(learning_rate=lr)
inps=apply_optimizer(fn,jnp.array([0.5,0]),opt,10)

fig,ax=plt.subplots()
plot_function(ax,fn)
ax.plot(inps[:,0],inps[:,1],"w.-");

In [82]:
lr=0.01
fn=elongated_bowl
opt=optax.sgd(learning_rate=lr)
inps=apply_optimizer(fn,jnp.array([5.,5]),opt,10)

fig,ax=plt.subplots()
plot_function(ax,fn,(-10,10),(-10,10))
ax.plot(inps[:,0],inps[:,1],"w.-");

Cette fonction est difficile à miniser car
*  une des variable la fait varier brusquement
*  une des variable la fait varier lentement

Ce genre de situation arrive fréquemment dans les problèmes d'apprentissage. Pour limiter ce phénomène, il faut "standardisez" les variables descriptives lors du prétraitement des données.

####  ♡

Faites varier le learning-rate. Observez.

## Méthodes de gradient élaborées



### SG avec Momentum

Le gradient est utilisé comme une accélération, et non comme une vitesse. Pour simuler une sorte de mécanisme de frottement et empêcher que l'élan ne devienne trop important, l'algorithme introduit un nouvel hyperparamètre $\mu$ le "momentum", qui doit être réglé entre 0 (frottement élevée) et 1 (aucune frottement). La valeur typique du momentum est de 0,9.




On veut trouver le minimum de $f(\theta)$. A l'étape $t=0$, on initialise $\theta_0$ aléatoirement et on pose $b_0=0$. Ensuite:
$$
g_t = \nabla_\theta f(\theta_{t-1})
$$

$$
b_t=\mu b_{t-1} + g_t
$$

Enfin, on moditie le paramètre que l'on veut optimiser:
$$
\theta_t = \theta_{t-1} - \gamma b_t
$$

Cela permet à cet optimiser de s'échapper des plateaux beaucoup plus rapidement que la Descente en Gradient.  Elle peut également aider à dépasser les optima locaux.

En raison du momentum, l'optimiseur peut dépasser un peu, puis revenir, dépasser à nouveau, et osciller comme cela plusieurs fois avant de se stabiliser au minimum. C'est l'une des raisons pour lesquelles il est bon d'avoir un peu de friction dans le système : cela élimine ces oscillations et accélère ainsi la convergence.

In [84]:
lr=0.005
fn=elongated_bowl
opt=optax.sgd(learning_rate=lr,momentum=0.9)
inps=apply_optimizer(fn,jnp.array([5.,5]),opt,10)

fig,ax=plt.subplots()
plot_function(ax,fn,(-10,10),(-10,10))
ax.plot(inps[:,0],inps[:,1],"w.-");

### A la main

In [93]:
def optimize_with_momentum(fn,inp0,lr,momentum,nb):

    inps=[inp0]
    inp=inp0
    b=0.

    for _ in range(nb):
        dinp=
        b=
        inp=
        inps.append(inp)

    return jnp.stack(inps)

In [94]:
lr=0.005
momentum=0.9
fn=elongated_bowl
inp0=jnp.array([5.,5])
inps=optimize_with_momentum(fn,inp0,lr,momentum,10)

fig,ax=plt.subplots()
plot_function(ax,fn,(-10,10),(-10,10))
ax.plot(inps[:,0],inps[:,1],"w.-");

### AdaGrad

Repensons au problème du bol allongé : La descente commence en suivant rapidement la pente la plus raide, puis descend lentement le fond de la vallée. Ce serait bien si l'algorithme pouvait détecter cela très tôt et corriger sa direction pour pointer un peu plus vers l'optimum global.
L'algorithme AdaGrad réalise cela en réduisant le vecteur gradient le long des dimensions les plus raides

On veut trouver le minimum de $f(\theta)$. A l'étape $t=0$, on initialise $\theta_0$ aléatoirement et on pose $s_0=0$. Ensuite:
$$
g_t = \nabla_\theta f(\theta_{t-1})
$$
$$
s_t=s_{t-1} + g^2_t
$$
Puis finalement:
$$
\theta_t = \theta_{t-1} - \gamma {g_t\over \sqrt{s_t} + \epsilon}
$$

 $\epsilon$ est un terme de lissage, par défaut $\epsilon=10^{-10}$.


On voit sur la formule que, cet algorithme diminue le taux d'apprentissage, mais il le fait plus rapidement pour les dimensions à forte pente que pour les dimensions à faible pente. C'est ce qu'on appelle un taux d'apprentissage adaptatif.

AdaGrad fonctionne souvent bien pour les problèmes quadratiques simples, mais malheureusement il s'arrête souvent trop tôt lors de l'apprentissage des réseaux de neurones. Le taux d'apprentissage est tellement réduit que l'algorithme finit par s'arrêter complètement avant d'atteindre l'optimum global.




### Adam

Adam, qui signifie "adaptive moment estimation", il combine les idées précédentes. Il y a 2 hyperparamètres $\beta_1$ et $\beta_2$ à choisir dans $[0,1]$. Auquel s'ajoute $\epsilon$ le paramètre de lissage dont la valeur par défaut est `1e-10`. Le paramètre $\gamma$ est le learning rate.





On veut trouver le minimum de $f(\theta)$. A l'étape $t=0$, on initialise $\theta_0$ aléatoirement et on pose $m_0=0$ et $s_0=0$. Ensuite:
$$
g_t = \nabla_\theta f(\theta_{t-1})
$$
Puis pour $t>0$:
$$
m_t=\beta_1 m_{t-1} + (1-\beta_1)g_t
$$
$$
s_t=\beta_2 s_{t-1} + (1-\beta_2)g^2_t
$$
Puis, on booste un peu les valeurs de $m_t$ et $s_t$ en faisant:
$$
m_t ←  {m_t \over 1-\beta_1^t}
$$
$$
s_t ←  {s_t \over 1-\beta_2^t}
$$
 Mais cela a un effet uniquement pour les $t$ petit (car les $\beta^t$ tendent vite vers 0).


Puis:
$$
\theta_t = \theta_{t-1} - \gamma {m_t \over \sqrt{s_t} + \epsilon}
$$

In [100]:
lr=0.5
fn=elongated_bowl
opt=optax.adam(learning_rate=lr)
inps=apply_optimizer(fn,jnp.array([5.,5]),opt,10)

fig,ax=plt.subplots()
plot_function(ax,fn,(-10,10),(-10,10))
ax.plot(inps[:,0],inps[:,1],"w.-");

### Régler le learning rate pour Adam

En fait, comme Adam est un algorithme au taux d'apprentissage adaptatif, il nécessite moins de réglage que les méthodes plus simples. Nous pouvons souvent utiliser la valeur par défaut $\ell =$ 1e-3. Mais il est bon de tester aussi 1e-1, 1e-2, 1e-4.

Il est bon aussi de faire descendre le learning rate très lentement au fur et à mesure de l'apprentissage pour descentre plus profondémment dans les puits de la loss.

On peut aussi utiliser la technique du recuit simuler: augmenter violemment le learning rate pour sortir d'un minimum local et aller explorer les allentours.

Bien entendu, on mémorise toujours les records successifs qu'on a attend pour pouvoir revenir à la fin au meilleurs endroit exploré; en apprentissage machine on appelle cela l'early stoping.


### En résumé

* Le momentum permet de sortir de minimums locaux, et aussi de ne pas trop rallentir sur des faux-plats.
* Les algorithmes adaptatifs évitent de se faire "aspirer" par la plus grande pente.
* Adam combine les deux idées précédentes.


Attention au learning_rate. Il n'a pas le même ordre de grandeur d'un optimiseur à l'autre.



## Défi prog

Dans le TP précédent, la fonction que l'on optimisée prenait comme argument un vecteur de dimension 2.

Est-t-il possible de travailler avec une fonction à 2 arguments scalaires à la place:

In [11]:
def fn(x,y):
    return x**2+y**2

Il faudrait adapter le plotter:

In [13]:
def plot_function_2_args(ax,fn,x_range=[-2,2],y_range=[-2,2]):

    x=jnp.linspace(x_range[0],x_range[1],100)
    y=jnp.linspace(y_range[0],y_range[1],100)

    fn_vv= vmap(vmap(fn,[None,0]),[0,None])

    Z=fn_vv(x,y)

    ax.pcolormesh(x,y,Z,cmap="jet",shading="gouraud")

⇑ Verstion plus courte et plus efficace que le plotter initial (moins de tenseurs intermédiaires).

In [14]:
fig,ax=plt.subplots()
plot_function_2_args(ax,fn)

Mais attention, par défaut, la fonction grad ne dérive qu'un seul argument

In [18]:
grad(fn)(5.,7.)

Il faut lui demander de dériver les 2:

In [19]:
grad(fn,argnums=[0,1])(5.,7.)

***A vous:*** Faites une optimisation SG avec momentum dans ce contexte. N'utilisez pas l'optimiseur d'`optax`.